In [ ]:
import yfinance as yf
import pandas as pd
import ta

def calculate_indicators(df):
    # Exponential Moving Average (EMA)
    df['EMA_12'] = ta.trend.EMAIndicator(df['Close'], window=12).ema_indicator()
    df['EMA_26'] = ta.trend.EMAIndicator(df['Close'], window=26).ema_indicator()

    # Moving Average Convergence Divergence (MACD)
    macd = ta.trend.MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()

    # Volume Weighted Average Price (VWAP)
    df['VWAP'] = ta.volume.VolumeWeightedAveragePrice(df['High'], df['Low'], df['Close'], df['Volume']).volume_weighted_average_price()

    # Relative Strength Index (RSI)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close']).rsi()

    # Stochastic Oscillator
    df['Stochastic'] = ta.momentum.StochasticOscillator(df['High'], df['Low'], df['Close']).stoch()

    # Average True Range (ATR)
    df['ATR'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close']).average_true_range()

    # Supertrend
    df['Supertrend'] = calculate_supertrend(df)

    # Calculate Bollinger Bands
    bollinger = ta.volatility.BollingerBands(df['Close'])
    df['BB_Upper'] = bollinger.bollinger_hband()
    df['BB_Lower'] = bollinger.bollinger_lband()

    # Calculate Fibonacci Levels
    df = calculate_fibonacci_levels(df)

    # Calculate Momentum manually
    period = 10
    df['Momentum'] = df['Close'].diff(periods=period)

    # Drop rows with NaN values
    df = df.dropna()
    return df

def calculate_supertrend(df, period=14, multiplier=3):
    # Calculate ATR
    df['ATR'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close'], window=period).average_true_range()

    # Initialize columns for Supertrend calculation
    df['Supertrend'] = pd.Series(index=df.index)
    df['Upper_Band'] = pd.Series(index=df.index)
    df['Lower_Band'] = pd.Series(index=df.index)
    
    # Calculate initial Supertrend values
    df['Upper_Band'] = df['Close'] + (df['ATR'] * multiplier)
    df['Lower_Band'] = df['Close'] - (df['ATR'] * multiplier)
    
    # Set initial Supertrend value
    df.loc[df.index[0], 'Supertrend'] = df.loc[df.index[0], 'Lower_Band']

    # Iterate over rows to calculate Supertrend
    for i in range(1, len(df)):
        if df.loc[df.index[i-1], 'Close'] > df.loc[df.index[i-1], 'Supertrend']:
            df.loc[df.index[i], 'Supertrend'] = max(df.loc[df.index[i], 'Lower_Band'], df.loc[df.index[i-1], 'Supertrend'])
        else:
            df.loc[df.index[i], 'Supertrend'] = df.loc[df.index[i], 'Upper_Band']
    
    return df['Supertrend']

def calculate_fibonacci_levels(df):
    # Assume we use the last 50 days to calculate Fibonacci levels
    window = 50
    df['Fibonacci_R1'] = df['Close'].rolling(window=window).apply(lambda x: max(x) - (0.236 * (max(x) - min(x))))
    df['Fibonacci_R2'] = df['Close'].rolling(window=window).apply(lambda x: max(x) - (0.382 * (max(x) - min(x))))
    df['Fibonacci_S1'] = df['Close'].rolling(window=window).apply(lambda x: min(x) + (0.236 * (max(x) - min(x))))
    df['Fibonacci_S2'] = df['Close'].rolling(window=window).apply(lambda x: min(x) + (0.382 * (max(x) - min(x))))
    return df

# Example usage
symbol = 'TCS.BO'
start_date = '2010-01-01'
end_date = '2024-07-31'
data = yf.download(symbol, start=start_date, end=end_date)
data = calculate_indicators(data)
data.to_csv(f"{symbol}_with_indicators.csv")

print(f"Data saved to {symbol}_with_indicators.csv")

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
import statsmodels.api as sm

# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load CSV data
df = pd.read_csv("tcs_data.csv")

# Define the trading environment
class TradingEnv:
    def __init__(self, df, initial_capital=10000):
        self.df = df
        self.current_step = 0
        self.action_space = [0, 1, 2]  # 0: Hold, 1: Buy, 2: Sell
        self.current_position = 0
        self.capital = initial_capital
        self.last_buy_price = 0

    def reset(self):
        self.current_step = 0
        self.current_position = 0
        self.capital = 10000
        self.last_buy_price = 0
        return self.get_state()

    def get_state(self):
        row = self.df.iloc[self.current_step]
        state = [
            row['EMA_12'],
            row['MACD'],
            row['VWAP'],
            row['RSI'],
            row['ATR'],
            row['Supertrend'],
            row['Upper_Band'],
            row['Lower_Band'],
            row['Fibonacci_R1'],
            row['Fibonacci_S1'],
            row['Momentum']
        ]
        return np.array(state)

    def step(self, action):
        row = self.df.iloc[self.current_step]
        stock_price = row['Close']
        reward = 0

        if action == 1:  # Buy
            if self.capital >= stock_price:
                shares_to_buy = self.capital // stock_price
                self.capital -= shares_to_buy * stock_price
                self.current_position += shares_to_buy
                self.last_buy_price = stock_price
            reward = 0

        elif action == 2:  # Sell
            if self.current_position > 0:
                shares_to_sell = self.current_position
                self.capital += shares_to_sell * stock_price
                profit = (stock_price - self.last_buy_price) * shares_to_sell
                reward = profit
                self.current_position = 0

        elif action == 0:  # Hold
            if self.current_position > 0:
                reward = (stock_price - self.last_buy_price) * self.current_position

        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        return self.get_state(), reward, done

# Define the DQN model
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Train the DQN agent
def train_agent(env, model, target_model, optimizer, criterion, episodes=100, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
    memory = deque(maxlen=2000)
    batch_size = 128
    target_update = 10
    rewards = []

    for episode in range(episodes):
        state = env.reset()
        state = torch.tensor(state, dtype=torch.float32).to(device)
        total_reward = 0
        done = False

        while not done:
            if random.random() < epsilon:
                action = random.choice(env.action_space)
            else:
                with torch.no_grad():
                    q_values = model(state)
                    action = torch.argmax(q_values).item()

            next_state, reward, done = env.step(action)
            next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
            total_reward += reward
            memory.append((state, action, reward, next_state, done))

            state = next_state

            if len(memory) > batch_size:
                batch = random.sample(memory, batch_size)
                train_step(batch, model, target_model, optimizer, criterion, gamma)

        rewards.append(total_reward)

        # Epsilon decay
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

        # Update the target model
        if episode % target_update == 0:
            target_model.load_state_dict(model.state_dict())

        print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")

    return rewards

def train_step(batch, model, target_model, optimizer, criterion, gamma):
    states, actions, rewards, next_states, dones = zip(*batch)

    states = torch.stack(states).to(device)
    actions = torch.tensor(actions, dtype=torch.int64).unsqueeze(1).to(device)
    rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1).to(device)
    next_states = torch.stack(next_states).to(device)
    dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1).to(device)

    q_values = model(states).gather(1, actions)
    next_q_values = target_model(next_states).max(1)[0].unsqueeze(1)
    target_q_values = rewards + (gamma * next_q_values * (1 - dones))

    loss = criterion(q_values, target_q_values)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluate the trained agent
def evaluate_agent(env, model):
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float32).to(device)
    total_reward = 0
    done = False

    while not done:
        with torch.no_grad():
            q_values = model(state)
            action = torch.argmax(q_values).item()

        next_state, reward, done = env.step(action)
        state = torch.tensor(next_state, dtype=torch.float32).to(device)
        total_reward += reward

    return total_reward

# Define and train ARIMA model
def arima_forecast(train_data, test_data):
    model = sm.tsa.ARIMA(train_data, order=(5, 1, 0))  # Adjust parameters as needed
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test_data))
    return forecast

# Define and train LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def lstm_forecast(train_data, test_data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(train_data.reshape(-1, 1))
    
    model = create_lstm_model((scaled_data.shape[1], 1))
    model.fit(scaled_data, train_data, epochs=100, batch_size=32, verbose=2)
    
    scaled_test_data = scaler.transform(test_data.reshape(-1, 1))
    forecast = model.predict(scaled_test_data)
    return forecast

# Performance metrics
def calculate_performance_metrics(returns):
    cumulative_returns = np.cumprod(1 + returns) - 1
    annualized_returns = np.mean(returns) * 252  # Assuming daily returns
    annualized_volatility = np.std(returns) * np.sqrt(252)
    sharpe_ratio = annualized_returns / annualized_volatility
    max_drawdown = np.min(cumulative_returns)  # Simplified calculation
    return annualized_returns, sharpe_ratio, max_drawdown

# Data preprocessing and train/test split
def preprocess_data(df):
    features = df[['EMA_12', 'MACD', 'VWAP', 'RSI', 'ATR', 'Supertrend', 'Upper_Band', 'Lower_Band', 'Fibonacci_R1', 'Fibonacci_S1', 'Momentum']].values
    prices = df['Close'].values
    return features, prices

def split_data(features, prices, split_ratio=0.8):
    split_idx = int(len(features) * split_ratio)
    train_features, test_features = features[:split_idx], features[split_idx:]
    train_prices, test_prices = prices[:split_idx], prices[split_idx:]
    return train_features, test_features, train_prices, test_prices

# Main execution
features, prices = preprocess_data(df)
train_features, test_features, train_prices, test_prices = split_data(features, prices)

# Train DQN agent
state_size = len(features[0])
action_size = 3
env = TradingEnv(df)

model = DQN(state_size, action_size).to(device)
target_model = DQN(state_size, action_size).to(device)
target_model.load_state_dict(model.state_dict())

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

rewards = train_agent(env, model, target_model, optimizer, criterion)

# Evaluate the trained RL agent
total_reward = evaluate_agent(env, model)
print(f"Total Reward of RL Agent: {total_reward}")

# Forecasting with ARIMA
arima_forecast_result = arima_forecast(train_prices, test_prices)
arima_metrics = calculate_performance_metrics(np.diff(arima_forecast_result))
print(f"ARIMA Model - Annualized Returns: {arima_metrics[0]}, Sharpe Ratio: {arima_metrics[1]}, Max Drawdown: {arima_metrics[2]}")

# Forecasting with LSTM
lstm_forecast_result = lstm_forecast(train_prices, test_prices)
lstm_metrics = calculate_performance_metrics(np.diff(lstm_forecast_result.flatten()))
print(f"LSTM Model - Annualized Returns: {lstm_metrics[0]}, Sharpe Ratio: {lstm_metrics[1]}, Max Drawdown: {lstm_metrics[2]}")


Using device: cuda
Episode 1/100, Total Reward: 33971.90554659999
Episode 2/100, Total Reward: 29200.91495499997
Episode 3/100, Total Reward: 31705.795959500043
Episode 4/100, Total Reward: 33618.51288409996
Episode 5/100, Total Reward: 80174.26941399983
Episode 6/100, Total Reward: 101350.74116229994
Episode 7/100, Total Reward: 38205.91336149995
Episode 8/100, Total Reward: 50002.48266579999
Episode 9/100, Total Reward: 40149.685786200105
Episode 10/100, Total Reward: 52756.66696830004
Episode 11/100, Total Reward: 45088.524713199964
Episode 12/100, Total Reward: 87684.40048450003
Episode 13/100, Total Reward: 101365.86669299997
Episode 14/100, Total Reward: 39359.78935539999
Episode 15/100, Total Reward: 15258.645888900057
Episode 16/100, Total Reward: 86235.67294699993
Episode 17/100, Total Reward: 37733.32113790002
Episode 18/100, Total Reward: 124701.59880690007
Episode 19/100, Total Reward: 93618.32988559986
Episode 20/100, Total Reward: 21850.88310529997
Episode 21/100, Total R

C:\Users\Asus\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
89/89 - 7s - 73ms/step - loss: 2357871.7500
Epoch 2/100
89/89 - 0s - 3ms/step - loss: 2337718.7500
Epoch 3/100
89/89 - 0s - 2ms/step - loss: 2309453.0000
Epoch 4/100
89/89 - 0s - 2ms/step - loss: 2292871.7500
Epoch 5/100
89/89 - 0s - 2ms/step - loss: 2280138.0000
Epoch 6/100
89/89 - 0s - 2ms/step - loss: 2268738.0000
Epoch 7/100
89/89 - 0s - 2ms/step - loss: 2258080.2500
Epoch 8/100
89/89 - 0s - 2ms/step - loss: 2247804.5000
Epoch 9/100
89/89 - 0s - 2ms/step - loss: 2237830.7500
Epoch 10/100
89/89 - 0s - 3ms/step - loss: 2228085.5000
Epoch 11/100
89/89 - 0s - 4ms/step - loss: 2218517.0000
Epoch 12/100
89/89 - 0s - 4ms/step - loss: 2209120.0000
Epoch 13/100
89/89 - 0s - 3ms/step - loss: 2199787.2500
Epoch 14/100
89/89 - 0s - 2ms/step - loss: 2190594.2500
Epoch 15/100
89/89 - 0s - 3ms/step - loss: 2181439.7500
Epoch 16/100
89/89 - 0s - 3ms/step - loss: 2172366.2500
Epoch 17/100
89/89 - 0s - 3ms/step - loss: 2163343.5000
Epoch 18/100
89/89 - 0s - 3ms/step - loss: 2154420.7500


C:\Users\Asus\AppData\Local\Temp\ipykernel_14904\2221612596.py:212: RuntimeWarning: invalid value encountered in scalar divide
  sharpe_ratio = annualized_returns / annualized_volatility
